In [102]:
from bs4 import BeautifulSoup

html_content = """
<div class="parent">
    <div class="child">Child 1</div>
    <div class="child">Child 2</div>
    <div class="child">Child 3</div>
</div>
"""

soup = BeautifulSoup(html_content, 'html.parser')

# 特定の親divを取得
parent_div = soup.find('div', class_='parent')

# 親divの中のすべての子divをリストとして取得
child_divs = parent_div.find_all('div', class_='child')

# 子divのテキストをリストとして取得
child_texts = [child.text for child in child_divs]

print(child_texts)

['Child 1', 'Child 2', 'Child 3']


In [108]:
import requests
from bs4 import BeautifulSoup
from retry import retry
import urllib
import time
import logging

import numpy as np
import re
import pandas as pd
from datetime import datetime

In [3]:
# SUUMOを東京都23区のみ指定して検索して出力した画面のurl(ページ数フォーマットが必要)
url = "https://suumo.jp/jj/bukken/ichiran/JJ010FJ001/?ar=030&bs=011&ta=13&jspIdFlg=patternShikugun&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13106&sc=13107&sc=13108&sc=13118&sc=13121&sc=13122&sc=13123&sc=13109&sc=13110&sc=13111&sc=13112&sc=13114&sc=13115&sc=13120&sc=13116&sc=13117&sc=13119&kb=1&kt=9999999&mb=0&mt=9999999&ekTjCd=&ekTjNm=&tj=0&cnb=0&cn=9999999&srch_navi={{2}}&page={}"


# リクエストがうまく行かないパターンを回避するためのやり直し
@retry(tries=3, delay=10, backoff=2)
def load_page(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    return soup

In [ ]:
# 面積を抽出する関数
def extract_area(text):
    match = re.search(r'(\d+(\.\d+)?)m2', text)
    if match:
        return match.group(1)
    return None

extract_area("50.00m2　（15.13m2）")

def extract_name(text):
    return text.split()[0]

extract_name("給田西住宅\u3000１号棟")

def extract_price(text):
    match = re.search(r'(\d+億)?\d+万円', text)
    if match:
        return match.group(0)
    return None

In [112]:
def calculate_age(built_date):
    # 現在の年と月を取得
    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month

    # built_dateを年と月に分割
    match = re.search(r'(\d+)年(\d+)月', built_date)
    if match:
        built_year = int(match.group(1))
        built_month = int(match.group(2))
    else:
        return None
    
    # 築年数を計算
    age_years = current_year - built_year
    age_months = current_month - built_month

    # 月の差がマイナスの場合、年から1を引き、月に12を足す
    if age_months < 0:
        age_years -= 1
        age_months += 12

    return age_years,age_months

calculate_age("2021年5月")

(3, 7)

In [51]:
soup = load_page(url.format(2))

In [ ]:
def get_apidata_suumo():
    # SUUMOを東京都23区のみ指定して検索して出力した画面のurl(ページ数フォーマットが必要)
    url = "https://suumo.jp/jj/bukken/ichiran/JJ010FJ001/?ar=030&bs=011&ta=13&jspIdFlg=patternShikugun&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13106&sc=13107&sc=13108&sc=13118&sc=13121&sc=13122&sc=13123&sc=13109&sc=13110&sc=13111&sc=13112&sc=13114&sc=13115&sc=13120&sc=13116&sc=13117&sc=13119&kb=1&kt=9999999&mb=0&mt=9999999&ekTjCd=&ekTjNm=&tj=0&cnb=0&cn=9999999&srch_navi={{2}}&page={}"
    ESTATES_MAX=30
    info={"name":[],
          "price":[],
          "address":[],
          "area":[],
          "age_year":[],
          "age_month":[],
          }

    for page in range(1,2):
        soup = load_page(url.format(page))
        estates_groups = soup.find("div",class_='property_unit_group')
        estates = estates_groups.find_all('div', class_='property_unit')

        for i in range(ESTATES_MAX):
            estate=estates[i].find_all('div', class_='dottable-line')

            
            name_text=estate[0].find_all("dd")[0].text
            info["name"].append(extract_name(name_text))
            price_text=estate[1].find_all("dd")[0].text
            info["price"].append(extract_price(price_text))
            info["address"].append(estate[2].find_all("dd")[0].text)
            area_text=estate[3].find_all("dd")[0].text
            info["area"].append(extract_area(area_text))
            built_date=estate[4].find_all("dd")[0].text
            age_years,age_months=calculate_age(built_date)

            


    return pd.DataFrame(info)

get_apidata_suumo()

,name,price,address,area,age
0,ロクタス,1億4700万円,東京都目黒区駒場１,92.36,2009年12月
1,コーポ東山,1100万円,東京都板橋区東山町,32.93,1975年4月
2,洗足シュロス,1400万円,東京都目黒区原町２-4-2,36.45,1972年6月
3,グリーンプラザ綾瀬II,1400万円,東京都足立区綾瀬６-7-15,37.27,1989年12月
4,豊栄アルベルゴ上野,1480万円,東京都台東区上野７-３－９,15.51,1980年3月
5,パークハイツ,1480万円,東京都板橋区高島平２,36.45,1978年6月
6,池袋シティハウス,1490万円,東京都板橋区中丸町3-1,18.62,1991年8月
7,鷺宮プリンスマンション,1500万円,東京都中野区鷺宮５-6-21,40.14,1977年9月
8,新菊川ビル,1540万円,東京都墨田区菊川１-17-17,37.37,1970年4月
9,久我山ロータリーマンション,1580万円,東京都杉並区久我山１-5-2,26.6,1976年4月


In [58]:
estates_groups = soup.find("div",class_='property_unit_group')
estates = estates_groups.find_all('div', class_='property_unit')

In [ ]:
for j in range(6):
    a=estates[1].find_all('div', class_='dottable-line')[j].find_all("dd")

    a_text=[item.text for item in a]

    print(a_text)
    print("--------------------------")

In [79]:
a='給田西住宅\u3000１号棟'

a.split("\u")


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 0-1: truncated \uXXXX escape (1715940244.py, line 3)

In [ ]:
child_divs = div_estates.find_all('div', class_='property_unit')
len(child_divs)

In [ ]:
child_texts = [child.text for child in child_divs]

In [ ]:
child_divs[0]